In [2]:
#!pip install transformers torch

In [6]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer


model_id = 'naver/splade_v2_max'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [7]:
text = "Information Retrieval course at the University of Southern Maine"
tokens = tokenizer(text, return_tensors='pt')
output = model(**tokens)
print(output)
print(output.logits.shape)
import torch

vec = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens.attention_mask.unsqueeze(-1),
dim=1)[0].squeeze()
# Get SPLADE Vector
print(vec.shape) #torch.Size([30522])


MaskedLMOutput(loss=None, logits=tensor([[[ -8.6903,  -8.6712,  -8.7282,  ...,  -7.7577,  -7.6470,  -8.4183],
         [-12.9129, -12.7483, -12.8099,  ..., -11.4523, -11.3861, -12.9494],
         [-10.7632, -10.4263, -10.6957,  ...,  -9.3885,  -9.1583, -12.1823],
         ...,
         [-13.8963, -14.0878, -14.1625,  ..., -11.4821, -12.5815, -12.8123],
         [-12.7274, -12.8395, -12.9904,  ..., -11.2488, -11.8553, -10.9621],
         [-10.6878, -10.6928, -10.7399,  ...,  -8.6930,  -8.7811,  -9.7368]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)
torch.Size([1, 11, 30522])
torch.Size([30522])


In [8]:
# extract non-zero positions
cols = vec.nonzero().squeeze().cpu().tolist()
print(len(cols))

# extract the non-zero values
weights = vec[cols].cpu().tolist()
# use to create a dictionary of token ID to weight
sparse_dict = dict(zip(cols, weights))
sparse_dict

# extract the ID position to text token mappings
idx2token = {
    idx: token for token, idx in tokenizer.get_vocab().items()
}


43


In [9]:
# map token IDs to human-readable tokens
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens


{'retrieval': 2.27,
 'information': 2.13,
 'southern': 2.11,
 'maine': 2.03,
 'university': 1.6,
 'course': 1.54,
 'retrieve': 1.35,
 'south': 1.34,
 'recall': 1.16,
 'retrieved': 1.14,
 'attend': 1.06,
 'massachusetts': 1.04,
 'bangor': 0.76,
 'remember': 0.73,
 'at': 0.7,
 'reference': 0.7,
 'uc': 0.7,
 'northern': 0.66,
 'library': 0.6,
 'info': 0.59,
 'school': 0.58,
 'open': 0.52,
 'degree': 0.48,
 'recognition': 0.47,
 'portland': 0.39,
 'key': 0.35,
 'stream': 0.34,
 'southwest': 0.33,
 'columbia': 0.3,
 'search': 0.27,
 'southeast': 0.26,
 'where': 0.23,
 'college': 0.2,
 'attended': 0.17,
 'u': 0.12,
 'data': 0.1,
 'processing': 0.08,
 'education': 0.05,
 'memory': 0.04,
 'guide': 0.03,
 'samuel': 0.02,
 'access': 0.01,
 'florida': 0.01}

In [10]:
import torch

text1 = "Information Retrieval course at the University of Southern Maine"
text2 = "Courses about search engines in Maine"
text3 = "Courses to avoid in computer science"
tokens1 = tokenizer(text1, return_tensors='pt')
tokens2 = tokenizer(text2, return_tensors='pt')
tokens3 = tokenizer(text3, return_tensors='pt')

output = model(**tokens1)
vec1 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens1.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

output = model(**tokens2)
vec2 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens2.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

output = model(**tokens3)
vec3 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens3.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

cos = torch.nn.CosineSimilarity(dim=0)
output = cos(vec1, vec2)
print(output)
output = cos(vec1, vec3)
print(output)

tensor([0.3208], grad_fn=<SumBackward1>)
tensor([0.0733], grad_fn=<SumBackward1>)
